<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d19e4b7f1b0df9dc82d62b3a690a5d2b8d16ac7785c307a019991d74b5a8b4ee
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 15:20:20
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: -96.06 K (-0.68%)
Current PnL: -21.19 L (-14.12%)
CY Booked + Current PnL: -9.29 L (-6.19%)
-------------------
Total profit:  1.24 L
Total loss:  -22.44 L
-------------------
Total Booked + Current PnL: 17.69 L (14.37%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.48%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.40 L (63.01%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,58.0,X-LC,3.69,214076.0,9576.0,13722.0,0.05,4.68,6.41,11.39,10.0,0.70,1.54,14.05,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,93.81,41.0,M-SC,1.26,83473.0,-17304.0,17396.0,0.31,-17.17,20.84,0.09,245.0,-0.99,0.60,9.67,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-1.36,51.0,X-MC,1.05,206925.0,15183.0,22576.0,-2.67,7.92,10.91,19.69,99.0,0.67,1.49,15.25,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,48.0,X-LC,2.97,214656.0,4290.0,24492.0,-0.41,2.04,11.41,13.68,37.0,0.18,1.54,18.90,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIS,528.00,2012.95,41.0,H-SC,3.81,85272.0,-25760.0,49364.0,2.81,-23.20,57.89,21.26,156.0,-0.52,0.61,15.21,OX40N,NTT,MISC
4,ANGELONE,3033.00,8.67,60.0,X-SC,10.72,192782.0,1776.0,55926.0,2.07,0.93,29.01,30.21,157.0,0.03,1.38,22.01,X40N,NTT,FINANCE
15,CAMPUS,393.00,-22.01,65.0,M-SC,2.39,162506.0,-5269.0,61119.0,1.78,-3.14,37.61,33.28,210.0,-0.09,1.17,25.95,XY24,NTT,FOOTWEAR
45,KPIGREEN,731.05,-4.80,45.0,H-SC,6.23,113488.0,-11809.0,70737.0,1.75,-9.42,62.33,47.03,141.0,-0.17,0.82,42.10,MH,ATH,POWER
31,HINDZINC,730.22,34.91,67.0,M-LC,8.32,220262.0,15186.0,97378.0,1.62,7.40,44.21,54.89,52.0,0.16,1.58,33.29,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GREENPANEL,537.0,221.06,51.0,M-SC,4.38,143423.0,-39655.0,118640.0,-5.13,-21.66,82.72,43.14,230.0,-0.33,1.03,34.91,XY24,NTT,MISC
20,COFFEEDAY,80.0,-45.67,50.0,L-SC,29.04,81792.0,-31757.0,71805.0,-3.42,-27.97,87.79,35.27,268.0,-0.44,0.59,100.19,XR,NTT,HOTELS
81,VALIANTORG,838.0,-287.15,27.0,H-SC,2.36,103972.0,-59633.0,180953.0,-3.38,-36.45,174.04,74.15,139.0,-0.33,0.75,33.28,XR,NTT,CHEMICALS
0,5PAISA,593.0,94.71,22.0,H-SC,8.32,132288.0,-86208.0,114403.0,-3.26,-39.46,86.48,12.90,153.0,-0.75,0.95,0.70,OX40N,NTT,FINANCE
46,LAOPALA,464.0,99.85,35.0,H-SC,1.20,91569.0,-39536.0,90782.0,-3.20,-30.16,99.14,39.09,142.0,-0.44,0.66,17.93,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.0,45.36,44.0,M-LC,6.33,204242.0,1072.0,41053.0,-0.43,0.53,20.10,20.74,55.0,0.03,1.47,6.10,XY25,NTT,FINANCE
85,WIPRO,420.0,-13.34,49.0,M-LC,5.83,151943.0,998.0,108457.0,-1.46,0.66,71.38,72.51,53.0,0.01,1.09,6.72,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,64.0,H-SC,11.33,131625.0,-10854.0,30379.0,-0.06,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,93.81,41.0,M-SC,1.26,83473.0,-17304.0,17396.0,0.31,-17.17,20.84,0.09,245.0,-0.99,0.60,9.67,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.97,221013.0,-48654.0,84979.0,-0.78,-18.04,38.45,13.47,138.0,-0.57,1.59,12.38,XY24,NTT,PAINTS
17,CERA,9475.0,-24.82,33.0,H-SC,1.82,137241.0,-38662.0,80684.0,-0.93,-21.98,58.79,23.89,149.0,-0.48,0.99,18.53,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,43.0,H-MC,6.82,101716.0,-30119.0,72340.0,-1.00,-22.85,71.12,32.03,98.0,-0.42,0.73,13.93,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,44.0,M-LC,6.33,204242.0,1072.0,41053.0,-0.43,0.53,20.10,20.74,55.0,0.03,1.47,6.10,XY25,NTT,FINANCE
59,SAIL,228.00,45.15,51.0,M-MC,12.66,228544.0,3582.0,164072.0,0.42,1.59,71.79,74.53,192.0,0.02,1.64,34.36,XY24,BTT,STEEL
31,HINDZINC,730.22,34.91,67.0,M-LC,8.32,220262.0,15186.0,97378.0,1.62,7.40,44.21,54.89,52.0,0.16,1.58,33.29,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,59.0,M-LC,9.39,159123.0,18283.0,26208.0,0.49,12.98,16.47,31.59,66.0,0.70,1.14,42.91,XY24,NTT,BANKS
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,151943.0,998.0,108457.0,-1.46,0.66,71.38,72.51,53.0,0.01,1.09,6.72,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANKINDIA,190.00,-24.22,68.0,H-MC,12.35,195049.0,15241.0,97173.0,0.04,8.48,49.82,62.52,88.0,0.16,1.40,45.45,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.32,220262.0,15186.0,97378.0,1.62,7.40,44.21,54.89,52.0,0.16,1.58,33.29,X5K,ATH,METALS
36,INDIAMART,4810.62,-54.45,43.0,H-SC,8.51,127158.0,3822.0,127807.0,0.33,3.10,100.51,106.72,119.0,0.03,0.91,27.20,AR,ATH,MISC
85,WIPRO,420.00,-13.34,49.0,M-LC,5.83,151943.0,998.0,108457.0,-1.46,0.66,71.38,72.51,53.0,0.01,1.09,6.72,XR,NTT,IT
84,WHIRLPOOL,2270.00,-48.24,32.0,M-SC,2.90,87502.0,-3996.0,82751.0,-0.24,-4.37,94.57,86.07,223.0,-0.05,0.63,27.07,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.77,52221.0,-12785.0,163713.0,-2.77,-19.67,313.50,232.18,198.0,-0.08,0.38,0.19,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229819.0,-17912.0,92962.0,-1.52,-7.23,40.45,30.30,92.0,-0.19,1.65,5.10,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-15.33,34.0,X-MC,5.15,178235.0,-22271.0,119417.0,-1.20,-11.11,67.00,48.45,90.0,-0.19,1.28,2.78,X40N,ATH,FINANCE
5,ASIANPAINT,3465.66,-18.29,34.0,X-LC,5.47,206087.0,-45681.0,98881.0,0.07,-18.14,47.98,21.13,27.0,-0.46,1.48,10.85,X40,ATH,PAINTS
22,DABUR,735.00,-12.48,35.0,X-MC,3.57,236704.0,-11364.0,119772.0,-0.34,-4.58,50.60,43.70,102.0,-0.09,1.70,9.20,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,HAVELLS,2069.16,-7.31,33.0,X-MC,0.90,229819.0,-17912.0,92962.0,-1.52,-7.23,40.45,30.30,92.0,-0.19,1.65,5.10,X40,ATH,ELECTRICAL
83,VOLTAS,1530.00,-1.36,51.0,X-MC,1.05,206925.0,15183.0,22576.0,-2.67,7.92,10.91,19.69,99.0,0.67,1.49,15.25,XY25,NTT,AC
72,TATAMOTORS,1065.00,-24.14,37.0,X-LC,1.17,239598.0,-36925.0,146993.0,-2.78,-13.35,61.35,39.81,54.0,-0.25,1.72,14.83,XY24,NTT,AUTO
64,SIEMENS,4671.50,-4.11,40.0,H-LC,1.29,154065.0,-32030.0,79513.0,-2.10,-17.21,51.61,25.51,15.0,-0.40,1.11,13.53,AR,ATH,ELECTRICAL
33,ICICIGI,2260.25,-21.04,37.0,X-MC,1.65,135679.0,-454.0,31586.0,-1.57,-0.33,23.28,22.86,91.0,-0.01,0.97,12.24,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.77,52221.0,-12785.0,163713.0,-2.77,-19.67,313.50,232.18,198.0,-0.08,0.38,0.19,XY24,NTT,MISC
82,VBL,671.64,-21.70,38.0,X-LC,10.48,282156.0,-33686.0,146354.0,-0.32,-10.67,51.87,35.67,5.0,-0.23,2.03,1.71,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.33,34.0,X-MC,5.15,178235.0,-22271.0,119417.0,-1.20,-11.11,67.00,48.45,90.0,-0.19,1.28,2.78,X40N,ATH,FINANCE
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG
21,COLPAL,3767.14,-4.98,38.0,X-MC,7.45,221990.0,-41375.0,154727.0,-0.35,-15.71,69.70,43.04,84.0,-0.27,1.59,3.06,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,32.0,X-SC,37.77,52221.0,-12785.0,163713.0,-2.77,-19.67,313.50,232.18,198.0,-0.08,0.38,0.19,XY24,NTT,MISC
55,RELAXO,1176.00,-40.46,37.0,X-SC,2.87,81207.0,-63953.0,139879.0,-0.61,-44.06,172.25,52.31,136.0,-0.46,0.58,8.70,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.29,38.0,X-SC,4.41,95251.0,-33419.0,78715.0,-1.83,-25.97,82.64,35.20,219.0,-0.42,0.68,9.33,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,51.0,X-SC,1.45,108102.0,6098.0,37825.0,0.77,5.98,34.99,43.06,122.0,0.16,0.78,24.12,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.21,108510.0,-19428.0,66560.0,-0.84,-15.19,61.34,36.84,143.0,-0.29,0.78,11.09,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.01,48.0,X-LC,12.18,282818.0,-63138.0,131369.0,-0.65,-18.25,46.45,19.73,1.0,-0.48,2.03,4.16,X40,ATH,IT
39,INFY,2275.00,-18.52,51.0,X-LC,7.63,317604.0,4716.0,166964.0,-1.57,1.51,52.57,54.87,3.0,0.03,2.28,8.26,X40,BTT,IT
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195282.0,-4856.0,25738.0,-0.86,-2.43,13.18,10.44,4.0,-0.19,1.40,3.00,X40,NTT,FMCG
82,VBL,671.64,-21.70,38.0,X-LC,10.48,282156.0,-33686.0,146354.0,-0.32,-10.67,51.87,35.67,5.0,-0.23,2.03,1.71,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,44.0,H-LC,7.45,247272.0,-14347.0,133552.0,-0.67,-5.48,54.01,45.57,7.0,-0.11,1.78,5.22,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.04,81792.0,-31757.0,71805.0,-3.42,-27.97,87.79,35.27,268.0,-0.44,0.59,100.19,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,42.0,H-SC,10.04,90600.0,-7380.0,28983.0,-0.92,-7.53,31.99,22.05,152.0,-0.25,0.65,31.69,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,64.0,H-SC,11.33,131625.0,-10854.0,30379.0,-0.06,-7.62,23.08,13.70,163.0,-0.36,0.95,55.35,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,16.41,146367.0,-2853.0,147040.0,0.25,-1.91,100.46,96.62,208.0,-0.02,1.05,58.40,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,60.0,H-SC,9.86,150338.0,-28733.0,94457.0,-2.40,-16.05,62.83,36.71,135.0,-0.30,1.08,49.17,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,16.41,146367.0,-2853.0,147040.0,0.25,-1.91,100.46,96.62,208.0,-0.02,1.05,58.40,XY25,NTT,FINANCE
68,SURYODAY,240.00,64.47,60.0,H-SC,9.86,150338.0,-28733.0,94457.0,-2.40,-16.05,62.83,36.71,135.0,-0.30,1.08,49.17,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.32,220262.0,15186.0,97378.0,1.62,7.40,44.21,54.89,52.0,0.16,1.58,33.29,X5K,ATH,METALS
63,SHALBY,327.00,1201.44,59.0,M-SC,22.44,176958.0,-4185.0,49654.0,1.31,-2.31,28.06,25.10,235.0,-0.08,1.27,41.81,XY24,NTT,HEALTHCARE
67,STARHEALTH,761.00,17.10,68.0,H-SC,7.85,260794.0,-20329.0,153190.0,0.14,-7.23,58.74,47.26,144.0,-0.13,1.87,39.73,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,43.84
2,50,75.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.99
MC    21.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.28
X40      14.67
XY25     11.94
XR       11.00
X40N     10.00
AR        8.18
OX40N     7.62
X200      1.77
X5K       1.58
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.66
X-LC    23.04
X-MC    15.90
M-SC    12.67
M-LC     5.28
H-LC     4.66
X-SC     4.58
H-MC     3.79
M-MC     1.64
L-SC     1.44
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.88,-6.19,41.19
IT,12.91,-17.06,80.45
FINANCE,12.05,-12.70,63.78
MISC,6.97,-18.40,84.00
BANKS,6.50,-10.23,68.23
PAINTS,5.55,-19.55,37.37
ELECTRICAL,5.32,-11.32,50.37
INSURANCE,3.80,-4.25,43.22
AUTO,3.47,-19.02,75.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3218150.0,22
XR,1377223.0,14
AR,1192496.0,9
X40,796237.0,10
X40N,636076.0,8
OX40N,586680.0,10
XY25,520009.0,8
SR,265602.0,2
X5K,97378.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3427442.0,25
M-SC,1279905.0,15
X-MC,1110797.0,11
X-LC,1035220.0,13
X-SC,542618.0,6
H-MC,351806.0,3
H-LC,292199.0,3
M-LC,273096.0,4
L-SC,260115.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1191708.0      6
           AR         814745.0      5
           XR         809820.0      7
M-SC       XY24       686391.0      6
X-MC       XY24       589124.0      4
X-LC       X40        492569.0      5
X-SC       X40N       300190.0      4
H-SC       OX40N      274830.0      4
           SR         265602.0      2
X-MC       X40        224953.0      4
X-LC       X40N       216469.0      3
           XY24       214641.0      2
H-LC       AR         213065.0      2
H-MC       XY24       182293.0      1
X-MC       XY25       177303.0      2
L-SC       XR         170714.0      2
M-SC       AR         164686.0      2
M-MC       XY24       164072.0      1
X-SC       XY24       163713.0      1
M-SC       OX40N      150109.0      4
           XY25       147040.0      1
           XR         131679.0      2
X-MC       X40N       119417.0      1
X-LC       XY25       111541.0      3
M-LC       XR         108457.0      1
           X5K         97378.0      1
H-MC       XR          97173.0      1
L-SC       OX40N       89401.0      1
H-LC       X200        79134.0      1
X-SC       X40         78715.0      1
H-MC       OX40N       72340.0      1
H-SC       MH          70737.0      1
L-MC       XR          59380.0      1
L-LC       XY25        43072.0      1
M-LC       XY25        41053.0      1
           XY24        26208.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
